Based on the following [notebook](https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb).

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="huggingface_hub")

In [2]:
!pip install corus
!pip install datasets
!pip install evaluate
!pip install razdel
!pip install seqeval



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 341.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=557ed708d08377a60ecdaa56051a73f245698e34a5e48955e7b44a61344108df
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In this notebook, we further fine-tune the model on the sequence labeling task of, namely, named entity recognition (aka named entity recognition, aka NER). We take a dataset of medical entities, but in general the pipeline is suitable for any task of identifying named entities in the text.


For speed, we will take a small BERT for Russian [rubert-tiny](https://huggingface.co/cointegrated/rubert-tiny2); if you take another, larger BERT-like model, the quality of NER may be higher, but the training and inference time will be longer.

In [3]:
model_checkpoint = "cointegrated/rubert-tiny2"
batch_size = 16

## Loading the dataset

For fine-tuning we take [Russian Drug Reaction Corpus](https://github.com/cimm-kzn/RuDReC): labelled corpus of Russian-language drug reviews.

Let's load it with the `corus` library.

In [4]:
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json

--2025-03-14 17:34:24--  https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json [following]
--2025-03-14 17:34:25--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1773014 (1.7M) [text/plain]
Saving to: ‘rudrec_annotated.json’

rudrec_annotated.js 100%[===================>]   1.69M  --.-KB/s    in 0.04s   

2025-03-14 17:34:25 (48.2 MB/s) - ‘rudrec_annotated.json’ saved [1773014/1773014]



In [5]:
from corus import load_rudrec
drugs = list(load_rudrec('rudrec_annotated.json'))
print(len(drugs))

4809


Document example:

In [6]:
drugs[0]

RuDReCRecord(
    file_name='172744.tsv',
    text='нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n',
    sentence_id=0,
    entities=[RuDReCEntity(
         entity_id='*[0]_se',
         entity_text='виферона',
         entity_type='Drugform',
         start=122,
         end=130,
         concept_id='C0021735',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[1]',
         entity_text='сыпью покрылся',
         entity_type='ADR',
         start=31,
         end=45,
         concept_id='C0015230',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[2]',
         entity_text='глаза опухли',
         entity_type='ADR',
         start=47,
         end=59,
         concept_id='C4760994',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[3]',
         entity_text

Let's look what entities we have.



* **DRUGNAME** Mentions of the brand name of a drug or product
ingredients/active compounds.
* **DRUGCLASS** Mentions of drug classes such as anti-inflammatory or
cardiovascular.
* **DRUGFORM** Mentions of routes of administration such as tablet
or liquid that describe the physical form in which
medication will be delivered into patient’s organism.
* **DI** Any indication/symptom that specifies the reason for
taking/prescribing the drug.
* **ADR** Mentions of untoward medical events that occur as a
consequence of drug intake and are not associated with
treated symptoms.
* **FINDING** Any DI or ADR that was not directly experienced by the
reporting patient or his/her family members, or related to
medical history/drug label, or any disease entities if the
annotator is not clear about type

Full description can be found in the [original paper](https://arxiv.org/abs/2004.03659).


In [7]:
from collections import Counter, defaultdict
type2text = defaultdict(Counter)
ents = Counter()
for item in drugs:
    for e in item.entities:
        ents[e.entity_type] += 1
        type2text[e.entity_type][e.entity_text] += 1

for k, v in ents.most_common():
    print(k, v)
    print(type2text[k].most_common(3))

DI 1401
[('простуды', 64), ('ОРВИ', 47), ('профилактики', 42)]
Drugname 1043
[('Виферон', 33), ('Анаферон', 25), ('Циклоферон', 24)]
Drugform 836
[('таблетки', 154), ('таблеток', 79), ('свечи', 63)]
ADR 720
[('аллергия', 16), ('слабость', 13), ('диарея', 12)]
Drugclass 330
[('противовирусный', 21), ('противовирусное', 18), ('противовирусных', 13)]
Finding 236
[('аллергии', 12), ('температуры', 6), ('сонливости', 5)]


In [8]:
drugs[0].text

'нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n'

Let's write a function that shifts entity markup to the word level. We use the [IOB](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)) notation to separate several entities of the same type in a row.

In [9]:
from razdel import tokenize

def extract_labels(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item.entities:
        e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e.entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e.entity_type

    return {'tokens': words, 'tags': word_labels}

In [10]:
print(extract_labels(drugs[0])['tags'])

['O', 'O', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'O', 'B-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'I-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Drugform', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [11]:
print(extract_labels(drugs[0])['tokens'])

['нам', 'прописали', ',', 'так', 'мой', 'ребенок', 'сыпью', 'покрылся', ',', 'глаза', 'опухли', ',', 'сверху', 'и', 'снизу', 'на', 'веках', 'высыпала', 'сыпь', ',', '(', '8', 'месяцев', 'сыну', ')', 'А', 'от', 'виферона', 'такого', 'не', 'было', '...', 'У', 'кого', 'ещё', 'такие', 'побочки', ',', 'отзовитесь', '!', '1', 'Чем', 'спасались', '?']


In [12]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for item in drugs]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

Data example

In [13]:
import pandas as pd
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
1150,"[Вот, и, сейчас, я, решила, применять, его, для, профилактики, себе, и, детям, .]","[O, O, O, O, O, O, O, O, B-DI, O, O, O, O]"
2606,"[Но, ,, если, заболеть, посерьезнее, ,, лично, я, как-то, сомневаюсь, в, действенном, лечении, этим, препаратом, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1407,"[Он, мне, раза, три, помог, ,, а, потом, перестал, помогать, .]","[O, O, O, O, O, O, O, O, O, O, O]"


Let's combine all labels in a list.

In [14]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

Let's store all object in a [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), native for `huggingface`.

In [15]:
from datasets import Dataset, DatasetDict

In [16]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 3847
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 962
    })
})

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

You can directly call this tokenizer on one sentence:

In [18]:
tokenizer("Привет, это одно предложение!")

{'input_ids': [2, 51343, 16, 2389, 13305, 24889, 5, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [19]:
tokenizer(["Привет", ",", "это", "одно", "предложение", "."], is_split_into_words=True)

{'input_ids': [2, 51343, 16, 2389, 13305, 24889, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [20]:
tokenized_input = tokenizer("Привет, это одно предложение!", is_split_into_words=False)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Привет', ',', 'это', 'одно', 'предложение', '!', '[SEP]']


In [21]:
tokenized_input = tokenizer(["Привет", ",", "это", "одно", "предложение", ",", "разбитое", "на", "слова", "."], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Привет', ',', 'это', 'одно', 'предложение', ',', 'разбито', '##е', 'на', 'слова', '.', '[SEP]']


In [22]:
tokenized_input["input_ids"]

[2, 51343, 16, 2389, 13305, 24889, 16, 52423, 626, 548, 7851, 18, 3]

To switch from the word level to the subword tokens level, we need to preprocess the texts one more time.

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [23]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 6, 7, 8, 9, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [24]:
example = ner_train[5]
print(example["tokens"])


['Мы', 'поменяли', 'место', 'жительства', 'и', 'перевели', 'дочь', 'в', 'школу', ',', 'которая', 'находится', 'ближе', 'к', 'дому', '.']


In [25]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

13 13


In [26]:
aligned_labels

[-100, 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', -100]

In [27]:
tokenized_input["input_ids"]

[2, 51343, 16, 2389, 13305, 24889, 16, 52423, 626, 548, 7851, 18, 3]

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the flag `label_all_tokens`.

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [28]:
def tokenize_and_align_labels(examples, label_all_tokens=False):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [29]:
tokenize_and_align_labels(ner_data['train'][22:23])

{'input_ids': [[2, 1041, 37038, 33265, 19106, 40305, 22018, 548, 22276, 320, 21538, 16, 47886, 548, 59614, 11137, 626, 56606, 700, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 1, -100, 7, 7, 7, 7, 0, 0, 0, 0, -100, -100, 0, -100, 0, -100]]}

In [30]:
ner_data['train'][22:23]

{'tokens': [['На',
   'следующее',
   'утро',
   'появились',
   'высыпания',
   'на',
   'лице',
   'и',
   'руках',
   ',',
   'похожие',
   'на',
   'комариные',
   'укусы',
   '.']],
 'tags': [['O',
   'O',
   'O',
   'O',
   'B-ADR',
   'I-ADR',
   'I-ADR',
   'I-ADR',
   'I-ADR',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']]}

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [31]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3847 [00:00<?, ? examples/s]

Map:   0%|          | 0/962 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [32]:
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

In [33]:
from transformers import AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [35]:
args = TrainingArguments(
    "ner",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_for_metrics=['inputs'],
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [36]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [37]:
from evaluate import load
metric = load("seqeval")

This metric takes list of labels for the predictions and references:

In [38]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'DI': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Drugform': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [39]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids
    predictions = np.argmax(predictions, axis=2)

    true_predictions = []
    true_labels = []
    for prediction, label in zip(predictions, labels):
        true_predictions.append([])
        true_labels.append([])
        for (p, l) in zip(prediction, label):
            if l != -100:
                true_predictions[-1].append(label_list[p])
                true_labels[-1].append(label_list[l])

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0.0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [40]:

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,  # Передайте tokenizer здесь
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [41]:
trainer.evaluate()

{'eval_loss': 2.6582188606262207,
 'eval_model_preparation_time': 0.0009,
 'eval_precision': 0.006283320639756283,
 'eval_recall': 0.06976744186046512,
 'eval_f1': 0.011528384279475981,
 'eval_accuracy': 0.03619708945852109,
 'eval_runtime': 1.2824,
 'eval_samples_per_second': 750.153,
 'eval_steps_per_second': 47.567}

Before training let's freeze all the model parameters except for the last layer, and see how well it learns.

In [42]:
for param in model.bert.parameters():
    param.requires_grad = False

In [43]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        print(param)

classifier.weight
Parameter containing:
tensor([[ 0.0357, -0.0445,  0.0160,  ..., -0.0219, -0.0253, -0.0093],
        [ 0.0244,  0.0368,  0.0232,  ..., -0.0212,  0.0259,  0.0283],
        [-0.0204,  0.0035,  0.0327,  ..., -0.0094, -0.0074, -0.0319],
        ...,
        [-0.0049,  0.0332,  0.0025,  ...,  0.0315,  0.0192, -0.0005],
        [ 0.0024,  0.0230,  0.0174,  ...,  0.0097,  0.0191,  0.0100],
        [ 0.0252, -0.0132,  0.0025,  ...,  0.0102,  0.0049, -0.0431]],
       device='cuda:0', requires_grad=True)
classifier.bias
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       requires_grad=True)


We can now finetune our model by just calling the `train` method:

In [44]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [45]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Precision,Recall,F1,Accuracy
1,No log,2.061143,0.000900,0.005447,0.021142,0.008662,0.618896
2,No log,1.585029,0.000900,0.004237,0.001057,0.001692,0.884022
3,2.071800,1.230263,0.000900,0.000000,0.000000,0.000000,0.900273
4,2.071800,0.984599,0.000900,0.000000,0.000000,0.000000,0.900864
5,1.228200,0.822904,0.000900,0.000000,0.000000,0.000000,0.900864
6,1.228200,0.720287,0.000900,0.000000,0.000000,0.000000,0.900864
7,0.817000,0.656881,0.000900,0.000000,0.000000,0.000000,0.900864
8,0.817000,0.619217,0.000900,0.000000,0.000000,0.000000,0.900864
9,0.649600,0.599446,0.000900,0.000000,0.000000,0.000000,0.900864
10,0.649600,0.593289,0.000900,0.000000,0.000000,0.000000,0.900864


TrainOutput(global_step=2410, training_loss=1.0901763852701147, metrics={'train_runtime': 27.1499, 'train_samples_per_second': 1416.95, 'train_steps_per_second': 88.767, 'total_flos': 24697669954800.0, 'train_loss': 1.0901763852701147, 'epoch': 10.0})

Not very well actually.


The model is underfitted: it looks like we should train more layers. Let's unfreeze them all (but perhaps it would be more correct to unfreeze only the top few), and fine-tune the model for 20 more epochs.

In [46]:
# разморозка
for param in model.parameters():
    param.requires_grad = True

In [47]:
args = TrainingArguments(
    "ner",
    eval_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_for_metrics=['inputs'],
)

In [48]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

In [49]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.319167,0.536998,0.268499,0.357999,0.919923
2,No log,0.278194,0.481212,0.419662,0.448334,0.927089
3,0.323500,0.253669,0.512528,0.475687,0.493421,0.932703
4,0.323500,0.237451,0.532397,0.521142,0.526709,0.937578
5,0.231700,0.225438,0.531469,0.562368,0.546482,0.941198
6,0.231700,0.215894,0.539359,0.586681,0.562025,0.942971
7,0.199300,0.206267,0.573171,0.596195,0.584456,0.945483
8,0.199300,0.202868,0.547641,0.625793,0.584114,0.945483
9,0.173100,0.195849,0.581016,0.640592,0.609351,0.948068
10,0.173100,0.193157,0.569302,0.646934,0.605641,0.947773


TrainOutput(global_step=4820, training_loss=0.1766850230110137, metrics={'train_runtime': 127.0982, 'train_samples_per_second': 605.359, 'train_steps_per_second': 37.923, 'total_flos': 49424108009730.0, 'train_loss': 0.1766850230110137, 'epoch': 20.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [50]:
trainer.evaluate()

{'eval_loss': 0.1792851686477661,
 'eval_precision': 0.6177861873226111,
 'eval_recall': 0.6902748414376322,
 'eval_f1': 0.6520219670494258,
 'eval_accuracy': 0.9537563714264609,
 'eval_runtime': 0.5489,
 'eval_samples_per_second': 1752.465,
 'eval_steps_per_second': 111.123,
 'epoch': 20.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [51]:
p = trainer.predict(tokenized_datasets["test"])


predictions, labels, inputs = p.predictions, p.label_ids, tokenized_datasets["test"]['input_ids']
predictions = np.argmax(p.predictions, axis=2)

# send only the first token of each word to the evaluation
true_predictions = []
true_labels = []
for prediction, label, tokens in zip(predictions, labels, inputs):
    true_predictions.append([])
    true_labels.append([])
    for (p, l, t) in zip(prediction, label, tokens):
        if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith('##'):
            true_predictions[-1].append(label_list[p])
            true_labels[-1].append(label_list[l])

results = metric.compute(predictions=true_predictions, references=true_labels, zero_division = 0.0)
results

{'ADR': {'precision': 0.34782608695652173,
  'recall': 0.37583892617449666,
  'f1': 0.3612903225806452,
  'number': 149},
 'DI': {'precision': 0.44416243654822335,
  'recall': 0.6481481481481481,
  'f1': 0.5271084337349398,
  'number': 270},
 'Drugclass': {'precision': 0.9047619047619048,
  'recall': 0.7808219178082192,
  'f1': 0.8382352941176471,
  'number': 73},
 'Drugform': {'precision': 0.8709677419354839,
  'recall': 0.9418604651162791,
  'f1': 0.9050279329608939,
  'number': 172},
 'Drugname': {'precision': 0.8015873015873016,
  'recall': 0.8938053097345132,
  'f1': 0.8451882845188284,
  'number': 226},
 'Finding': {'precision': 1.0,
  'recall': 0.017857142857142856,
  'f1': 0.03508771929824561,
  'number': 56},
 'overall_precision': 0.6177861873226111,
 'overall_recall': 0.6902748414376322,
 'overall_f1': 0.6520219670494258,
 'overall_accuracy': 0.9537563714264609}

In [52]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [53]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-ADR,B-DI,B-Drugclass,B-Drugform,B-Drugname,B-Finding,I-ADR,I-DI,I-Drugclass,I-Drugform,I-Drugname,I-Finding
O,12061,15,40,3,18,15,0,9,33,0,0,1,0
B-ADR,35,77,34,0,0,1,0,2,0,0,0,0,0
B-DI,53,14,189,2,2,5,0,1,4,0,0,0,0
B-Drugclass,5,0,9,57,0,2,0,0,0,0,0,0,0
B-Drugform,7,0,0,0,163,2,0,0,0,0,0,0,0
B-Drugname,4,0,3,0,2,217,0,0,0,0,0,0,0
B-Finding,10,13,31,0,0,1,1,0,0,0,0,0,0
I-ADR,60,8,5,0,0,0,0,66,30,0,0,0,0
I-DI,70,5,20,0,0,0,0,3,78,0,0,0,0
I-Drugclass,0,0,0,1,0,0,0,0,0,0,0,0,0


In [54]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

('ner_bert.bin/tokenizer_config.json',
 'ner_bert.bin/special_tokens_map.json',
 'ner_bert.bin/vocab.txt',
 'ner_bert.bin/added_tokens.json',
 'ner_bert.bin/tokenizer.json')

# Model application

In [55]:
import torch

In [56]:

text = ' '.join(ner_test[4]['tokens'])  # 458 is example of a difficult text
text

'Охотно применяю его при борьбе с насморком , что в моем случае явление очень частое .'

In [57]:
tokens = tokenizer(text, return_tensors='pt').to(model.device)

with torch.no_grad():
    pred = model(**tokens)
pred.logits.shape

torch.Size([1, 21, 13])

In [58]:
indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
for t, idx in zip(token_text, indices):
    print(f'{t:15s} {label_list[idx]:10s}')

[CLS]           O         
Охо             O         
##тно           O         
применя         O         
##ю             O         
его             O         
при             O         
борьбе          O         
с               O         
насмор          B-DI      
##ком           I-DI      
,               O         
что             O         
в               O         
моем            O         
случае          O         
явление         O         
очень           O         
частое          O         
.               O         
[SEP]           O         


Easier solution: huggingface `pipeline`.

In [59]:
from transformers import pipeline

In [60]:
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

Device set to use cuda:0


In [61]:
print(text)
print(pipe(text))

Охотно применяю его при борьбе с насморком , что в моем случае явление очень частое .
[{'entity_group': 'DI', 'score': 0.453896, 'word': 'насморком', 'start': 33, 'end': 42}]


# Home assignment
In the seminar, we apply tinyBERT to the Named Entity Recognition task in drug description texts. We first freeze the main body of the model and train 10 epoch, then unfreeze it and train during the 20 epochs.
Now try to freeze the first module of the BERT, and learn a full model for 30 epochs.

What score on the test set will you get in this case? Write 2-3 sentences commenting on the results in the markdown.

As a solution, send one ZIP archive (only ZIP format is supported) which contains:

1) A modified notebook version

2) A screenshot with the final result and your comment

